# Feature Selection  -- featurelearni Tanlab olish bosqichi

# Feature Selection 3 ta qadamga bo`linadi 
- 1-tur = Feature Filtering
- 2-tur = Feature Wrapper
- 3-tur = Feature Embedded

# Feature Filtering



-  statistical qiymatlarga ko`ra featurelarni tanlanadi
- featureni qanchalik target qiymatga bog`liqligiga ko`ra tanlanadi // agarda 1 bo`lsa = Strong Relationship // agarda 0  bo`lsa = independet relationship - bog`liqlik yo`q


# Feature Filtering Metodlari : Correlation, Variance threshold, Anova

# Feature Filteringda 2 xil natija ko`p kuzatilishi mumkin

- 1- Multicollinearity -- juda yuqori bo`gliq bo`lgan feature
- 2- LoW Varience -- juda kam tasir qilayotgan feature 

# Feature Filtering ni + va  - tamonlari

- "+" lari 
- tez ishlaydi  (algaritm va  training qilish shart emas )
- high -dimensional datasetlar bilan yaxshi ishlaydi 
- overfittingni kamaytiradi 


- "-" lari 
- barcha featurelar alohida alohida baholanadi
- kafolatli emas  

# Feature Wrapper 

# Bu usulda - ML model training qilgandan so`ng featurelarni baxolaydi 

# Baxolash mezoni - qanchalik model accuracyga qo`shgan xissasiga ko`ra tanlanadi - relative contributions

# Feature Wrapperning bir nechta metodlari bor -- RFE, Forward Selection, Backward Elimination

# Feature Wrapperning ishlashi-- Eng avval : turli xil datasetlar bilan ko`p marotaba tarining qiladi  , Keyin esa: eng yuqori natija bergan subsetlar eng yuqori featurelar sifatida qabul qilinadi

# Feature  Wrapperning + va - tamonlari
- "+" lari 
- featurelarni alohidadan ko`ra guruh bilan yaxshi natija berishini xisobga oladi
- odatda fiteringdan  ko`ra yaxshiroq natija beradi 

- "- " lari 
- ko`p vaqt oladi 
- overfittingga moyil 

# Feature Embedded usuli

# bu usulda model training davomida qo`shgan hissasiga qarab  featurelarni tanlab oladi 

# ishlash tartibi - feature  qanchalik  impurity ni kamaytirganiga qarab aniqlanadi (% larda o`lchanadi )
- impurity bu modeldagi shovqin 

# Feature Embedded Metodlari: Lasso Regression, gradient Boosting, Random Forest 

# Feature Embedded ning + va  -  tamonlari 
- "+" lari 
- featurelar training davomida eng yaxshilari tanlanadi
- featurearni birgalikda muhimligio inobatga olinadi 
- katta datasetlar bilan yaxshi ishlaydi 

- "-" lari 
- interpretability qiyin
- scaling uchun bazi metodlar sensitive  

# XULOSA

# Qaysi xolatda qaysi usul bilan  ishlanadi

- FILTERING     

- high dimensional datasets
- tezkor feature selection uchun 
- model trainingsiz va oshiqcha vaqtlarsiz 


- WRAPPER 

- kichik yoki o`rta hajmli datasetlar
- eng yuqori natija kerak bo`lsa 
- ko`p vaqt bor bo`lsa 


- EMBEDDED 

-  avtomatik feature selection 
- o`rtacha dataset 
- xar bir algoritm uchun  maxsus featurelar kerak bo`lsa 

# Korrelyatsiya - bu ikki o`zgaruvchining biur biriga qanchalik bo`gliqligini ko`rsatuvchi o`lchov 

# Range -1 dan +1 gacha 

agarda +1 bo`glanish bo`lsa bu ijobiy bog`lanish deyiladi 
agarda -1 bo`glanish bo`lsa bu salbiy bog`lanish deyiladi 
agarda 0 bo`lsa bog`lanish yo`q deyiladi

# Highlty correlated features - bir biriga juda yaqin bo`glangan featurelardir (r >= 0.8)

# Higly correlated bo`lsa nima muammolar kelib chiqadi
- 1-Multicollinearity hosil bo`ladi 
- 2- modelning interpretatsiyasi qiyinlashadi
- 3- Bazi ML algoritmlari (Linear Regression, Logistic regression, uchun xatolikni oshiradi)

# Low Correlated - bir biriga juda kam tasir qilayotgan featurelar

# Low Correlated da Multicollinearity muammosi yo`q 

# Yuqoridagilardan kleib chiqgan holda ML algoritmga qarab qaror qabul qiling 
- Tree based (Random Forest, XGBoost ) -->    multicollinearity muammo emas 
- Linear Algoritmlar (Logistic Regression Linear regression) --->  multicollinearity muammo qiladi 